## Assignment: Data Splitting

1. Split your data into a training and test set
2. Use cross validation to fit a model on all numeric features. Report r2 values for each validation fold.
3. Fit your model on all of your training data and score on the test dataset. 

In [0]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np

computers = pd.read_csv("../Data/Computers.csv")

computers.tail()

### Test Data Split

In [0]:
from sklearn.model_selection import train_test_split


features = ["speed", "hd", "ram", "screen", "ads", "trend"]

X = sm.add_constant(computers[features])
y = computers["price"]

# Test Split
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=12345)

### Cross Validation Loop

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae



kf = KFold(n_splits=5, shuffle=True, random_state=2023)

# Create a list to store validation scores for each fold
cv_lm_r2s = []
cv_lm_mae = []

# Loop through each fold in X and y
for train_ind, val_ind in kf.split(X, y):
    # Subset data based on CV folds
    X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
    X_val, y_val = X.iloc[val_ind], y.iloc[val_ind]
    # Fit the Model on fold's training data
    model = sm.OLS(y_train, X_train).fit()
    # Append Validation score to list 
    cv_lm_r2s.append(r2(y_val, model.predict(X_val),))
    cv_lm_mae.append(mae(y_val, model.predict(X_val),))

print("All Validation R2s: ", [round(x, 3) for x in cv_lm_r2s])
print(f"Cross Val R2s: {round(np.mean(cv_lm_r2s), 3)} +- {round(np.std(cv_lm_r2s), 3)}")

print("All Validation MAEs: ", [round(x, 3) for x in cv_lm_mae])
print(f"Cross Val MAEs: {round(np.mean(cv_lm_mae), 3)} +- {round(np.std(cv_lm_mae), 3)}")

### Model Fit on All Training Data

In [0]:
model = sm.OLS(y, X).fit()

model.summary()

### Score on Test Data

In [0]:
print(f"Test R2: {r2(y_test, model.predict(X_test))} ")
print(f"Test MAE: {mae(y_test, model.predict(X_test))}")